<a href="https://colab.research.google.com/github/Syamabbas/Google-Collab/blob/main/Auto_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
import re
from datetime import timedelta
from collections import defaultdict

df = pd.read_excel("/content/drive/MyDrive/Google Collab/Data Cleansing Sym.xlsx")

In [37]:
REFERENCE_DATE = pd.Timestamp("2025-12-01")

def convert_relative_time(text, ref_date=REFERENCE_DATE):
    if pd.isna(text):
        return None

    text = str(text).lower().strip()
    text = text.replace("diedit", "").strip()

    special_map = {
        "sebulan lalu": ("bulan", 1),
        "seminggu lalu": ("minggu", 1),
        "setahun lalu": ("tahun", 1)
    }

    if text in special_map:
        unit, value = special_map[text]
    else:
        match = re.search(r"(\d+)\s+(menit|jam|hari|minggu|bulan|tahun)\s+lalu", text)
        if not match:
            return None
        value = int(match.group(1))
        unit = match.group(2)

    if unit == "menit":
        return ref_date - timedelta(minutes=value)
    elif unit == "jam":
        return ref_date - timedelta(hours=value)
    elif unit == "hari":
        return ref_date - timedelta(days=value)
    elif unit == "minggu":
        return ref_date - timedelta(weeks=value)
    elif unit == "bulan":
        return ref_date - pd.DateOffset(months=value)
    elif unit == "tahun":
        return ref_date - pd.DateOffset(years=value)

    return None

In [38]:
df['tanggal_review'] = df['Waktu'].apply(convert_relative_time)
df['tanggal_review'] = pd.to_datetime(df['tanggal_review']).dt.date

In [52]:
# LOAD KAMUS TYPO
sheet_id = "12ypDLePiU0f-50wsfsQ8N21aWLjmeb8q5k9L8tn4YIU"

def load_gsheet(sheet_name):
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return pd.read_csv(url)

# url = f"https://docs.google.com/spreadsheets/d/12ypDLePiU0f-50wsfsQ8N21aWLjmeb8q5k9L8tn4YIU/gviz/tq?tqx=out:csv&sheet=Kamus_Typo"

typo_df = load_gsheet("Kamus_Typo")

# pastikan lowercase & tidak ada NaN
typo_df['Typo'] = typo_df['Typo'].astype(str).str.lower().str.strip()
typo_df['Correction'] = typo_df['Correction'].astype(str).str.lower().str.strip()

# convert ke dictionary
typo_dict = dict(zip(typo_df['Typo'], typo_df['Correction']))

print(f"Total typo loaded: {len(typo_dict)}")

Total typo loaded: 115


In [40]:
# PATTERN NORMALIZATION
patterns = {
    r'(\d+)\s*hri': r'\1 hari',
    r'(\d+)\s*hr': r'\1 hari',
    r'(\d+)\s*d': r'\1 hari'
}

In [41]:
# CLEANING FUNCTIONS
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)  # huruf berulang
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def normalize_pattern(text):
    for p, rpl in patterns.items():
        text = re.sub(p, rpl, text)
    return text

def normalize_typo(text):
    for typo, correct in typo_dict.items():
        text = re.sub(rf'\b{re.escape(typo)}\b', correct, text)
    return text

In [42]:
# APPLY TO DATAFRAME
df['clean_text'] = df['Komentar'].apply(clean_text)
df['clean_text'] = df['clean_text'].apply(normalize_pattern)
df['clean_text'] = df['clean_text'].apply(normalize_typo)

In [43]:
# LOAD KAMUS SENTIMENT FROM GOOGLE SHEET
sentiment_df = load_gsheet("Kamus_Sentiment")

# normalisasi
sentiment_df['keyword'] = sentiment_df['keyword'].astype(str).str.lower().str.strip()
sentiment_df['sentiment'] = sentiment_df['sentiment'].astype(str).str.lower().str.strip()

# pisahkan positive & negative
positive_words = sentiment_df[sentiment_df['sentiment'] == 'positif']['keyword'].tolist()
negative_words = sentiment_df[sentiment_df['sentiment'] == 'negatif']['keyword'].tolist()

print("Positive words:", len(positive_words))
print("Negative words:", len(negative_words))

In [44]:
def auto_label(text, pos_words, neg_words):
    pos_count = 0
    neg_count = 0

    for word in pos_words:
        if word in text:
            pos_count += 1

    for word in neg_words:
        if word in text:
            neg_count += 1

    if pos_count > neg_count:
        return "positif"
    elif neg_count > pos_count:
        return "negatif"
    else:
        return "netral"

In [ ]:
df['label'] = df['clean_text'].apply(
    lambda x: auto_label(x, positive_words, negative_words)
)

df[['Komentar', 'label']]

In [46]:
df_ml = df[df['label'] != 'netral'].copy()

In [47]:
def auto_label_with_score(text, pos_words, neg_words):
    pos_count = sum(1 for w in pos_words if w in text)
    neg_count = sum(1 for w in neg_words if w in text)
    score = abs(pos_count - neg_count)

    if pos_count > neg_count:
        return "positif", score
    elif neg_count > pos_count:
        return "negatif", score
    else:
        return "netral", 0


In [ ]:
df[['label','confidence']] = df['clean_text'].apply(
    lambda x: pd.Series(
        auto_label_with_score(x, positive_words, negative_words)
    )
)

df_ml = df[df['confidence'] >= 1].copy()

In [ ]:
positive_words = sorted(positive_words, key=len, reverse=True)
negative_words = sorted(negative_words, key=len, reverse=True)

In [50]:
from google.colab import files
# Simpan ke Excel
file_name = "hasil_auto_labeling.xlsx"
df_ml.to_excel(file_name, index=False)

# Download
files.download(file_name)
print("File berhasil dibuat dan sedang diunduh!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File berhasil dibuat dan sedang diunduh!
